# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import re

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [24]:
def movie_title_url_actor():
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res =  urllib.request.Request(url)
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    rank = soup.find_all("dl", class_="lst_dsc") # 제목 찾기
    rank2 = soup.find_all("dl", class_="info_txt1") # 배우들 찾기

    for i, context in enumerate(rank[:5]):
        title = context.find("a").get_text()
        print(str(i+1) + ".영화제목: " + title)
        detail = rank2[i].find_all("span", class_="link_txt")
        name_group = detail[2].find_all("a")
        print("주연배우: ", end=" ")
        for name in name_group[:3]:
            print(name.get_text(),end=" ")
        print("")

In [25]:
movie_title_url_actor()

1.영화제목: 인비저블맨
주연배우:  엘리자베스 모스 
2.영화제목: 지푸라기라도 잡고 싶은 짐승들
주연배우:  전도연 정우성 배성우 
3.영화제목: 1917
주연배우:  조지 맥케이 딘-찰스 채프먼 
4.영화제목: 정직한 후보
주연배우:  라미란 김무열 나문희 
5.영화제목: 작은 아씨들
주연배우:  시얼샤 로넌 엠마 왓슨 플로렌스 퓨 


### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [30]:
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd

In [33]:
def get_grade(movie_name):
    search = movie_name
    url = 'https://movie.naver.com/movie/running/current.nhn'
    driver = webdriver.Chrome('./chromedriver.exe') 
    driver.get(url)
    sleep(5)
    SCROLL_PAUSE_TIME = 1.5
    
    driver.find_element_by_id('ipt_tx_srch').send_keys(search) # search에 담긴 영화를 입력한다.
    driver.find_element_by_class_name('btn_srch').click() # 그리고 버튼을 클릭!
    driver.find_element_by_partial_link_text(search).click() # 검색된 내용 중 해당 영화 페이지 들어가기
    html = driver.page_source
    
    soup = BeautifulSoup(html, 'html.parser')
    rank = soup.find_all("div", class_="star_score")
    for context in rank[:3]:
        score = context.find_all('em')
        for i in score:
            print(i.get_text(), end='')
        print("")

In [34]:
get_grade('지푸라기라도 잡고 싶은 짐승들')

8.24
6.71
6.67


### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [39]:
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd

In [50]:
def get_reviews(movie_name):
    search = movie_name
    url = 'https://movie.naver.com/movie/running/current.nhn'
    driver = webdriver.Chrome('./chromedriver.exe') 
    driver.get(url)
    sleep(5) 
    SCROLL_PAUSE_TIME = 1.5
    
    driver.find_element_by_id('ipt_tx_srch').send_keys(search) # 영화이름 입력하기
    driver.find_element_by_class_name('btn_srch').click() # 검색 클릭!
    driver.find_element_by_partial_link_text(search).click() # 영화 상세 페이지 들어가기
    driver.find_element_by_xpath('//*[@id="movieEndTabMenu"]/li[5]/a').click() # 영화의 리뷰가 담긴 페이지로 이동
    
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=179181&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
    # 위 url은 크롬 개발자도구에서 네트워크 패킷 보고 가져왔습니다.. 코드로 가져오는걸 못하겠어서 ㅠ
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    reple = soup.find('div', class_='score_result')
    comment = reple.find_all('li')
    result = []
    
    for i in comment:
        
        # 평점, 리뷰, 리뷰작성자를 구분지어 ['평점','리뷰','작성자'] 형태로 저장하였습니다.
        temp = []
        star = i.find('em')
        print('평점'+ star.get_text())
        context = i.find('div', class_='score_reple')
        print('리뷰: '+ context.find('span').get_text().strip())
        name = context.find('dt')
        print('이름: ' + name.find('span').get_text())
        temp.append(star.get_text())
        temp.append(context.find('span').get_text().strip())
        temp.append(name.find('span').get_text())
        result.append(temp)
        print(' ')
        
    # 다음 페이지로 이동!
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=179181&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2'
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    reple = soup.find('div', class_='score_result')
    
    comment = reple.find_all('li')
    for i in comment:
        temp = []
        star = i.find('em')
        print('평점'+ star.get_text())
        context = i.find('div', class_='score_reple')
        print('리뷰: '+ context.find('span').get_text().strip())
        name = context.find('dt')
        print('이름: ' + name.find('span').get_text())
        temp.append(star.get_text())
        temp.append(context.find('span').get_text().strip())
        temp.append(name.find('span').get_text())
        result.append(temp)
        print(' ')
    
    
    return result

In [51]:
reviews = get_reviews('지푸라기라도 잡고 싶은 짐승들')
reviews

평점10
리뷰: 난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 지푸라기
이름: bohemian(mabu****)
 
평점10
리뷰: 전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
이름: 최정규(cjg4****)
 
평점10
리뷰: 8명의 배우가 모두 주인공 같은 느낌.
이름: 달다(fxko****)
 
평점9
리뷰: 개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
이름: 써니(tlag****)
 
평점1
리뷰: 아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽부터 함? 백번양보해서 시사회때 봤다 쳐도 이렇게나 많이 봤다고? 죄다 똑같은 말투에? 음원이고 영화고 조작질 역겹다 진짜
이름: 어쩌라고(dpfk****)
 
평점10
리뷰: 연출, 연기, 스토리 모두 대박...무조건 보세요.
이름: 까칠소녀(oper****)
 
평점9
리뷰: 관람객
이름: Cjin(dlck****)
 
평점10
리뷰: 스토리가 짱짱하네요~ 심리적인 긴장감을 잘 살린 영화인것 같네요~ 인기좀 끌듯...
이름: trau****
 
평점9
리뷰: 한국식 피칠갑을 한 타란티노 영화
이름: hojo****
 
평점10
리뷰: 관람객
이름: 꽁끼(bamb****)
 
평점9
리뷰: 관람객
이름: haeunnnnn(0_80****)
 
평점10
리뷰: 관람객
이름: 시무룩(simu****)
 
평점10
리뷰: 전도연을 위한, 전도연에 의한 영화! 데뷔작이라고는 믿을수 없는 연출력에놀랐다~
이름: Linus(getu****)
 
평점10
리뷰: 이 캐스팅 너무 마음에 든다.영화보고나서도 할말이 많아지는 영화
이름: myd5q3ji7(i2g1****)
 
평점9
리뷰: 솔직히 이 영화 돈주고 볼만합니다ㅎㅎ
이름: 파리투나잇(lgxe****)
 
평점10
리뷰: 진짜 전도연은 대체불가다.
이름: 김연아이유(revo****)
 
평점10
리뷰: 관람객
이름: 꿈꾸는피터팬(dowk****)
 
평

[['10',
  '난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 지푸라기',
  'bohemian(mabu****)'],
 ['10', '전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...', '최정규(cjg4****)'],
 ['10', '8명의 배우가 모두 주인공 같은 느낌.', '달다(fxko****)'],
 ['9', '개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ', '써니(tlag****)'],
 ['1',
  '아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽부터 함? 백번양보해서 시사회때 봤다 쳐도 이렇게나 많이 봤다고? 죄다 똑같은 말투에? 음원이고 영화고 조작질 역겹다 진짜',
  '어쩌라고(dpfk****)'],
 ['10', '연출, 연기, 스토리 모두 대박...무조건 보세요.', '까칠소녀(oper****)'],
 ['9', '관람객', 'Cjin(dlck****)'],
 ['10', '스토리가 짱짱하네요~ 심리적인 긴장감을 잘 살린 영화인것 같네요~ 인기좀 끌듯...', 'trau****'],
 ['9', '한국식 피칠갑을 한 타란티노 영화', 'hojo****'],
 ['10', '관람객', '꽁끼(bamb****)'],
 ['9', '관람객', 'haeunnnnn(0_80****)'],
 ['10', '관람객', '시무룩(simu****)'],
 ['10', '전도연을 위한, 전도연에 의한 영화! 데뷔작이라고는 믿을수 없는 연출력에놀랐다~', 'Linus(getu****)'],
 ['10', '이 캐스팅 너무 마음에 든다.영화보고나서도 할말이 많아지는 영화', 'myd5q3ji7(i2g1****)'],
 ['9', '솔직히 이 영화 돈주고 볼만합니다ㅎㅎ', '파리투나잇(lgxe****)'],
 ['10', '진짜 전도연은 대체불가다.', '김연아이유(revo****)'],
 ['10', '관람객', '꿈꾸는피터팬(dowk**

### 4. 저장하기

In [52]:
def save(context):
    f = open('data.txt', 'w')
    f.write(context)
    f.close

In [53]:
import pandas as pd
data = pd.DataFrame(reviews)

In [54]:
data.columns = ['star', 'review', 'writer']
data.to_csv('크롤링결과.csv', encoding='cp949')